# Optimizaton Modeling with Gurobi and Python

This notebook demonstrates how we can use Gurobi optimization software to solve optimization problems in Python. The following table of contents includes links to the various content.


# Table of Contents
<a id="Table_of_Contents"> </a>

1. [Introduction](#Introduction)<br>
2. [Model Construction](#Model_Construction)<br>
3. [Modifying Model Parameters](#Modifying_Model_Parameters)<br>
4. [Providing_Initial_Solutions (Warmstarts)](#Providing_Initial_Solutions)<br>
5. [Adding Constraints *on the fly* with Lazy Constraint Callbacks](#Lazy_Constraints)<br>
6. [Implementing Custom Cutting Planes](#Custom_Cutting_Planes)<br> 
    
#### Disclaimer

This notebook is by no means a comprehensive resource for Gurobi's Python API. Also, it is important to realize that the Python language and the available packages will continue to evolve. That being said, the objects, functions, and methods described in this notebook may one day change. If changes occur, areas of this notebook that use deprecated features may cease to work and will need to be revised or omitted. Additional details on Gurobi's Python API, as well as example code, can be found at http://www.gurobi.com.


## Introduction
<a id="Introduction"> </a>

Gurobi is named for its founders: Zonghao **Gu**, Edward **Ro**thberg and Robert **Bi**xby. Bixby was also the founder of CPLEX, one of Gurobi's competitors, while Rothberg and Gu led the CPLEX development team for nearly a decade. Assuming that you have 1) installed Gurobi, 2) obtained a valid license, and 3) performed the necessary steps to install the Gurobi Python module, the following code block imports the gurobipy module, along with the numpy and datetime modules, that will allow us to access model problems in Python and solve them with Gurobi.

[Back to Table of Contents](#Table_of_Contents)<br>

In [1]:
import numpy as np

import datetime

from gurobipy import *

We will use a variant of the knapsack problem to demonstrate the use of Gurobi. The knapsack problem is a combinatorial optimization problem that may be stated as:

> "Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible."

The knapsack problem that we will consider is the 0-1 knapsack problem, which restricts the number of each item that may be included in the solution to zero or one. Specifically, given a set of $n$ items numbered from $1$ up to $n$, each with a weight $\displaystyle w_{i}$ and value $\displaystyle v_{i}$, and a maximum weight capacity $W$, the 0-1 Knapsack problem can be formulated as:
<center>
Maximize $\displaystyle \sum _{i=1}^{n}v_{i}X_{i}$<br>
subject to $\displaystyle \sum _{i=1}^{n}w_{i}X_{i}\leq W$, and $\displaystyle X_{i}\in \{0,1\},$ 
</center>

where $X_{i}$ is a binary variable that denotes whether or item $i$ is included in the knapsack. Informally, the problem seeks to maximize the sum of the values for the items incuded in the knapsack while ensuring that the total weight of the included items is less than or equal to the knapsack's capacity.

We will begin by demonstrating how to construct the 0-1 Knapsack model using Gurobi. The following code block randomly generates weight and benefit values for a 1000-item problem instance and sets the available capacity to 25% of the product of the mean item weight and the number of items.

[Back to Table of Contents](#Table_of_Contents)<br>

In [2]:
# Specify the number of items to generate
num_items = 1000

# Set the seed value for numpy's random number generator
np.random.seed(40)

# Generate a random sample of integer item weights between 5 and 20
weights = np.random.randint(5,21,num_items)

# Generate a random sample of integer item values between 5 and 20
values = np.random.randint(5,21,num_items)

# Creae a list with the item indices using a list comprehension
# Note that the first index is zero!
items = [i for i in range(len(weights))]

# Specify the available capacity as 25% of the product of
# the mean item weight and the number of items
available_capacity = 0.25*np.mean(weights)*num_items

## Model Construction
<a id="Model_Construction"> </a>

The first thing that we need to do is create a Gurobi `Model` object. The following code block creates a `Model` object named `model`.

[Back to Table of Contents](#Table_of_Contents)<br>

In [3]:
model = Model('Knapsack')

Academic license - for non-commercial use only


Next, we will define our decision variables. For our problem, we need binary variables for each of the $n$ items. The variables are created in the following code block. Note that the `vtype` argument may be set to:

1. `GRB.CONTINUOUS` to model continuous variables, 
2. `GRB.BINARY` to model binary variables, 
3. `GRB.INTEGER`, to model general integer variables,
4. `GRB_SEMICONT`, to model semi-continuous variables, or 
5. `GRB_SEMIINT` to model semi-integer variables.

[Back to Table of Contents](#Table_of_Contents)<br>

In [4]:
# The following lines of code show how to add the variables as binary variables
assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

model.update()

The following code block shows how we can build expressions that can be used to construct objective functions and constraints. In particular, the following code block deines the objective for our 0-1 Knapsack problem. Although Gurobi offers many different ways to constructs the stated model objects, I find this to be relatively simple and easier to align with a given mathematical formulation. 

[Back to Table of Contents](#Table_of_Contents)<br>

In [5]:
# Create an empty linear expression (LinExpr()) object
temp_sum = LinExpr()

# For all items
for i in items:
    
    # Add the product of the binary decision variable for the item and its benefit to the linear expression
    temp_sum.add(assignment[i]*values[i])

# Set the objective of the model to maximize the constructed linear expression    
model.setObjective(temp_sum, GRB.MAXIMIZE)

# Update the model to incoporate the objective
model.update()

The following code block uses a similar technique to add the problem constraints.

[Back to Table of Contents](#Table_of_Contents)<br>

In [6]:
# Create an empty linear expression (LinExpr()) object
temp_sum = LinExpr()

# For all items
for i in items:
    
    # Add the product of the binary decision variable for the item and its weight to the linear expression
    temp_sum.add(assignment[i]*weights[i])
    
# Add a constraint that requires the constructed linear expression to be less than or equal to the available capacity     
model.addConstr(temp_sum <= available_capacity)

# Update the model to incoporate the new constraints
model.update()

We have now defined our model, and can use `model.optimize` to solve the problem instance.

[Back to Table of Contents](#Table_of_Contents)<br>

In [7]:
model.optimize()

Optimize a model with 1 rows, 1000 columns and 1000 nonzeros
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 3e+03]
Found heuristic solution: objective 3146.0000000
Presolve removed 0 rows and 751 columns
Presolve time: 0.00s
Presolved: 1 rows, 249 columns, 249 nonzeros
Variable types: 0 continuous, 249 integer (21 binary)

Root relaxation: objective 5.577857e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5577.85714    0    1 3146.00000 5577.85714  77.3%     -    0s
H    0     0                    5577.0000000 5577.85714  0.02%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 5577 3146 

Optimal

The output of our optimization model shows that an optimal solution was found in a small fraction of a second. This solution results in a total value of 5,557 for the assigned objects. We can access the optimal value of our decision variables by using the `x` attribute of our `assignment` object. The following code block shows how we can use this attribute to construct a list that includes the assigned items. The first five assigned items are printed.

[Back to Table of Contents](#Table_of_Contents)<br>

In [8]:
# Create an empty list
assigned_items = []

# For all items
for current_item in range(len(assignment)):
    
    # If the value of the binary variable for the item is greater than 0.5 in the optimal solution
    if assignment[current_item].x > 0.5:
        
        # Append the variable name to the list
        assigned_items.append(assignment[current_item].varname)

# Print the first five items in the list
print(f'The first five items in the optimal solution are:\n {assigned_items[:5]}')

The first five items in the optimal solution are:
 ['Item[0]', 'Item[2]', 'Item[4]', 'Item[5]', 'Item[6]']


The following cell puts everything together, and shows how to use the `datetime` package to capture the model build and solve time.

[Back to Table of Contents](#Table_of_Contents)<br>

In [9]:
# Calculate start time
model_build_starttime = datetime.datetime.now()

# Create model
model = Model('Knapsack')

# Create binary variables for assignments
assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

# Define and set objective
temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])    
model.setObjective(temp_sum, GRB.MAXIMIZE)


# Define and add capacity constraint
temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])   
model.addConstr(temp_sum <= available_capacity)

# Optimize model
model.optimize()

# Create a list of items included in knapsack
assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

# Compute solution time        
model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

# Print solution information
print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The first five items in the optimal solution are:\n {assigned_items[:5]}')

Optimize a model with 1 rows, 1000 columns and 1000 nonzeros
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 3e+03]
Found heuristic solution: objective 3146.0000000
Presolve removed 0 rows and 751 columns
Presolve time: 0.00s
Presolved: 1 rows, 249 columns, 249 nonzeros
Variable types: 0 continuous, 249 integer (21 binary)

Root relaxation: objective 5.577857e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5577.85714    0    1 3146.00000 5577.85714  77.3%     -    0s
H    0     0                    5577.0000000 5577.85714  0.02%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 5577 3146 

Optimal

## Modifying Model Parameters
<a id="Modifying_Model_Parameters"> </a>

In this section, you will learn how to adjust parameters that control the optimization process. Lists and details for parameters that can be specified may be found by looking at the documentation available at www.gurobi.com. The following code block solves a 50-item 0-1 knapsack instance using the code written in the previous section. We will build upon this example throughout this section.

**Note: In the following code blocks, comments are only used to indicate areas of new content.**

[Back to Table of Contents](#Table_of_Contents)<br>

In [10]:
weights = [94, 506, 416, 992, 649, 237, 457, 815, 446, 422, 791, 359, 667,
           598, 7, 544, 334, 766, 994, 893, 633, 131, 428, 700, 617, 874,
           720, 419, 794, 196, 997, 116, 908, 539, 707, 569, 537, 931, 726,
           487, 772, 513, 81, 943, 58, 303, 764, 536, 724, 789]

values = [403, 886, 814, 1151, 983, 629, 848, 1074, 839, 819,
            1062, 762, 994, 950, 111, 914, 737, 1049, 1152, 1110,
            973, 474, 824, 1013, 963, 1101, 1024, 816, 1063, 575,
            1153, 447, 1117, 910, 1017, 931, 909, 1126, 1027, 871,
            1052, 891, 375, 1131, 318, 705, 1048, 908, 1026, 1061]

items = [i for i in range(len(weights))]

available_capacity = 900

model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])    
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])   
model.addConstr(temp_sum <= available_capacity)

model.optimize()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The first five items in the optimal solution are:\n {assigned_items[:5]}')

Optimize a model with 1 rows, 50 columns and 50 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [7e+00, 1e+03]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+02, 9e+02]
Found heuristic solution: objective 2029.0000000
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 1 rows, 37 columns, 37 nonzeros
Variable types: 0 continuous, 37 integer (37 binary)

Root relaxation: objective 3.278920e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3278.91983    0    1 2029.00000 3278.91983  61.6%     -    0s
H    0     0                    2890.0000000 3278.91983  13.5%     -    0s
H    0     0                    3014.0000000 3278.91983  8.79%     -    0s
     0     0 3070.48649    0    2 3014.00000 3070.48649  1.87%     -    0s

Cut

As can be seen in the previous code block, Gurobi is able to find the optimal solution for this instance quickly and uses proprietary heuristic methods during the solution process instead of pure mathematical programming techniques. Suppose that for a particular problem we are attempting to solve, the heuristics are not as effective and tend to slow down the solution process. The following code block shows how we can modify our code to disable the use of heuristics during the optimization process.

[Back to Table of Contents](#Table_of_Contents)<br>

In [11]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')

# Sets the value of the Heuristics parameter to 0, i.e., no heuristics
model.setParam('Heuristics', 0)

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])    
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])   
model.addConstr(temp_sum <= available_capacity)

model.optimize()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The first five items in the optimal solution are:\n {assigned_items[:5]}')

Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Optimize a model with 1 rows, 50 columns and 50 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [7e+00, 1e+03]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+02, 9e+02]
Presolve removed 0 rows and 11 columns
Presolve time: 0.00s
Presolved: 1 rows, 39 columns, 39 nonzeros
Variable types: 0 continuous, 39 integer (39 binary)

Root relaxation: objective 3.278920e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3278.91983    0    1          - 3278.91983      -     -    0s
H    0     0                    3014.0000000 3278.91983  8.79%     -    0s
     0     0 3070.48649    0    2 3014.00000 3070.48649  1.87%     -    0s

Cutting planes:
  Cover: 1

Explo

Instead of using `model.setParam()`, we can also change the value of model parameters using *attribute* syntax. This is demonstrated in the following code block which is equivalent to the previous.

[Back to Table of Contents](#Table_of_Contents)<br>

In [12]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')

# Use attribute-style syntax to diable heuristics
model.params.Heuristics = 0

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])    
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])   
model.addConstr(temp_sum <= available_capacity)

model.optimize()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The first five items in the optimal solution are:\n {assigned_items[:5]}')

Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Optimize a model with 1 rows, 50 columns and 50 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [7e+00, 1e+03]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+02, 9e+02]
Presolve removed 0 rows and 11 columns
Presolve time: 0.00s
Presolved: 1 rows, 39 columns, 39 nonzeros
Variable types: 0 continuous, 39 integer (39 binary)

Root relaxation: objective 3.278920e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3278.91983    0    1          - 3278.91983      -     -    0s
H    0     0                    3014.0000000 3278.91983  8.79%     -    0s
     0     0 3070.48649    0    2 3014.00000 3070.48649  1.87%     -    0s

Cutting planes:
  Cover: 1

Explo

Oftentimes, when we are trying to solve a very hard model, we may wish to set a criterion (such as a time limit or bound on the optimality gap) that allows us to terminate the optimization before finding an optimal solution. The following code block shows how to specify a value for the gap between the incumbent solution and the best bound that we can use for such termination purposes. Specifically, we specify that such a gap value of 10% is sufficient. Thus, the optimization terminates as soon as a feasible solution is found with a corresponding gap value that is less than or equal to 10%.

[Back to Table of Contents](#Table_of_Contents)<br>

In [13]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')
model.params.Heuristics = 0

# Set gap value for termination to 10% 
model.setParam('MIPGap',0.10)

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])    
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])   
model.addConstr(temp_sum <= available_capacity)

model.optimize()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The first five items in the optimal solution are:\n {assigned_items[:5]}')

Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Optimize a model with 1 rows, 50 columns and 50 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [7e+00, 1e+03]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+02, 9e+02]
Presolve removed 0 rows and 11 columns
Presolve time: 0.00s
Presolved: 1 rows, 39 columns, 39 nonzeros
Variable types: 0 continuous, 39 integer (39 binary)

Root relaxation: objective 3.278920e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3278.91983    0    1          - 3278.91983      -     -    0s
H    0     0                    3014.0000000 3278.91983  8.79%     -    0s

Cutting pla

## Providing Initial Solutions (Warmstarts)
<a id="Providing_Initial_Solutions"> </a>

Substantial improvement in the solution time for a hard MIP is possible if a high-quality initial solution is provided at the onset of optimization. Such starting solutions can be obtained via a heuristic or some other approach. Gurobi provides a simple way to provide initial solutions through the `start` attribute of variable objects. The following code block shows how this attribute may be used to provide the initial solution $X_{1} = X_{2} = 1$ for the 0-1 Knapsack instance defined in the previous section.

[Back to Table of Contents](#Table_of_Contents)<br>

In [14]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')
model.params.Heuristics = 0

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

# Specify an initial value of 1 for items 0 and 1.
# Essntially, this defines an initial solution that includes items 1 and 2
assignment[0].start = 1.0
assignment[1].start = 1.0

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])    
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])   
model.addConstr(temp_sum <= available_capacity)

model.optimize()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The first five items in the optimal solution are:\n {assigned_items[:5]}')

Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Optimize a model with 1 rows, 50 columns and 50 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [7e+00, 1e+03]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+02, 9e+02]

MIP start produced solution with objective 2567 (0.01s)
Loaded MIP start with objective 2567

Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 1 rows, 37 columns, 37 nonzeros
Variable types: 0 continuous, 37 integer (37 binary)

Root relaxation: objective 3.278920e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3278.91983    0    1 2567.00000 3278.91983  27.7%     -    0s
H    0     0                    3014.0000000 3254.93103  7.99%     -    0s
     0     0 3

As you can see, the optimization initializes using the solution we defined. It is worth noting that we did not have to define a complete solution, i.e., values for all decision variables, because Gurobi will attempt to complete partial solutions. However, it is advised to provide as complete of a solution as possible becuase Gurobi may give up on trying to complete a particla solution if the process is taking too long.

[Back to Table of Contents](#Table_of_Contents)<br>

## Adding Constraints *on the fly* with Lazy Constraint Callbacks
<a id="Lazy_Constraints"> </a>

Sometimes, the number of constraints required to model a problem can be prohibitively large, e.g., the TSP with all sub-tour elimination constraints added at the onset of optimization. However, Gurobi provides `Lazy Constraints` for adding constraints in an *as needed* fashion during an optimization run. Specifically, the user defines a set of lazy constraints, but they are not initially added to the optimization model. Whenever a new incumbent solution is found, the solver checks to see whether or not the solution violates any constraints in the pool of lazy constraints. If so, the violated constraints are added and the optimization resumes. Repeasting this process, the optimization only add the constraints that are necessary to obtain the optimal solution.

Gurobi implements lazy constraints using callbacks. A callback is essentially a function that can interact with an optimization model. A complete understanding of callbacks is beyond the scope of this tutorial, but additional information can be found at www.gurobi.com. The following code block generates a 100-item instance that we will use for this demonstration.

[Back to Table of Contents](#Table_of_Contents)<br>

In [15]:
num_items = 100
np.random.seed(40)

weights = np.random.randint(5,20,num_items)
values = np.random.randint(5,20,num_items)
items = [i for i in range(len(weights))]

available_capacity = 0.25*np.mean(weights)*num_items

The following code block solves the problem without using lazy constraints. Note that the `Heuristics` have been disabled.

[Back to Table of Contents](#Table_of_Contents)<br>

In [16]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')

model.setParam('Heuristics', 0)

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item ")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])
model.addConstr(temp_sum <= available_capacity)

model._vars = assignment
model.optimize()

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The optimal solution chooses:\n {assigned_items}')

Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Optimize a model with 1 rows, 100 columns and 100 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 3e+02]
Presolve removed 0 rows and 15 columns
Presolve time: 0.00s
Presolved: 1 rows, 85 columns, 85 nonzeros
Variable types: 0 continuous, 85 integer (71 binary)

Root relaxation: objective 4.800769e+02, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  480.07692    0    1          -  480.07692      -     -    0s
     0     0  479.74074    0    2          -  479.74074      -     -    0s
     0     0  479.66667    0    1          -  479.66667      -     -    0s
     0     0  479.60000    0  

Note that the solution to the problem includes several consecutive items. For example, items 2 and 3, items 5 and 6, and items 6 and 7. Suppose that there was some reason that consecutively indexed items should not be selected in an optimal solution. Although for the small problem that we are considering, it would be simple to add such constraints at the onset, we will use lazy constraints within a callback to incorporate the additional requirements as needed. The following code block defines the callback function that we will use. 

[Back to Table of Contents](#Table_of_Contents)<br>

In [17]:
# Note that the function expects 2 arguments:
# 1) a 'model' object,
# 2) a 'where' object that specifies the status of the optimization
def prohibit_consecutive_assignments(model, where):
    
    # If a new MIP incumbent has been found
    if where == GRB.Callback.MIPSOL:
        
        # Get the values of the decision variables in the solution
        vals = model.cbGetSolution(model._vars)
        
        # for all items i in 1..(n-1)
        for i in range(len(model._vars.keys())-1):
            j = i + 1
            
            # check to see if coonsecutives items are included
            if (vals[i] > 0.5) & (vals[j] > 0.5):
                
                # if so, add a constraint that prohibits the assignments
                model.cbLazy(model._vars[i] + model._vars[j] <= 1.0)

To use the callback, we must set the `LazyConstraints` parameter to one, define a `model._vars` attribute to pass the decision variable values to the callback, and add the defined callback function as an argument to the `model.optimize()` call.

[Back to Table of Contents](#Table_of_Contents)<br>

In [18]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')
model.setParam('Heuristics', 0)

# Modify LazyConstraints parameter
model.params.LazyConstraints = 1

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item ")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])
model.addConstr(temp_sum <= available_capacity)

# Define model._vars attribute
model._vars = assignment

# Pass callback function as an argument to the optimization
model.optimize(prohibit_consecutive_assignments)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The optimal solution chooses:\n {assigned_items}')

Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 1 rows, 100 columns and 100 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 3e+02]
Presolve time: 0.00s
Presolved: 1 rows, 100 columns, 100 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 4.800769e+02, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  480.07692    0    1          -  480.07692      -     -    0s
     0     0  479.80000    0    1          -  479.80000      -     -    0s
     0     0  479.66667    0    1          -  479.666

As you can see, the output from the last optimization shows that several lazy constraints were added. Also, note that no consecutively indexed items are selected in the solution.

[Back to Table of Contents](#Table_of_Contents)<br>

## Implementing Custom Cutting Planes 
<a id="Custom_Cutting_Planes"> </a>

In this section, we will look at the use of callbacks to add cutting planes during an optimization. Recall that **the lazy constraint method added constraints that defined the feasible region of the associated problem**. Essentially, the approach added these constraints as needed instead of adding them all at the onset of the optimization. **In contrast, cutting planes are typically applied at nodes of the branch & bound tree and can change the feasible space of the continuous relaxation without ruling out any feasible integer solution that the rest of the model permits.** You can read more about the difference between user cuts and lazy constraints at https://www.ibm.com/support/knowledgecenter/SSSA5P_12.6.1/ilog.odms.cplex.help/CPLEX/UsrMan/topics/progr_adv/usr_cut_lazy_constr/04_diffs.html and https://orinanobworld.blogspot.com/2012/08/user-cuts-versus-lazy-constraints.html (both accessed 1/2/2019).

The following code block specifies the data for a 7-item instance that we use to demonstrate the use of cutting planes.

[Back to Table of Contents](#Table_of_Contents)<br>

In [19]:
weights = [11, 6, 6, 5, 5, 4, 1]
values = [7, 6, 5, 4, 3, 2, 1]
items = [0, 1, 2, 3, 4, 5, 6]

available_capacity = 20

Before continuing, it should be noted that today's state of the art solvers implement very sophisticated (and usually good) features for quickly determining high-quality feasible solutions. These techniques includes presolving modifications, heuristic procedures, methods for generating several classes of cutting planes, and methods to distribute the tasks associated with solving an optimization problem accross multiple computer cores. **If you want to use callbacks to implement a custom cutting plane procedure, you must disable several of these features including presolve modifications, and distributed computing**. 

Although the decision to use custom cutting planes requires disabling several features, it can be worthwhile. To demonstrate the impact of cutting planes, the following code block solves the defined problem instance without used several of the features noted in the previous paragraph.

[Back to Table of Contents](#Table_of_Contents)<br>

In [20]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')
model.setParam('Heuristics', 0)
model.Params.Presolve = 0
model.Params.Threads = 1
model.Params.Cuts = 0

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])
model.addConstr(temp_sum <= available_capacity)

model._vars = assignment
model.optimize()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The optimal solution chooses:\n {assigned_items}')

Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 1 rows, 7 columns and 7 nonzeros
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Variable types: 0 continuous, 7 integer (7 binary)

Root relaxation: objective 1.727273e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   17.27273    0    1          -   17.27273      -     -    0s
     0     0   17.27273    0    1          -   17

Even with the selected features disabled, Gurobi is able to determine the optimal solution quickly. However, not the number of notes that Gurobi needed to process.

We will use Gurobi callbacks to add *cover inequalities* during the solution process. **Note: By default, Gurobi attempts to identify valid cover inequalities and add them as part of its cut generation features. The purpose of doing this manually is purely for demonstration purposes.** Although you will learn more about cover inequalities when studying integer programming, at a high level, cover inequality cuts typically seek to identify minimal cardinality subsets of decision variables that *just* violate any problem constraints. Once identified, a cut is added that limits the sum of the decision variables in the subset.  

The following code block uses the `itertools` package to define a function that generates all subsets of a given set.

[Back to Table of Contents](#Table_of_Contents)<br>

In [21]:
import itertools
def findsubsets(S,m):
    return set(itertools.combinations(S, m))

In practice, the process of identifying valid cover inequalities can be cumbersome. Thus, we are going to use an extremely naive, and less efficient, method for the purpose of demonstration. Essentially, we will identify all subsets of the items whose total weight exceeds the available capacity. The following code block constructs a list of all such subsets.

[Back to Table of Contents](#Table_of_Contents)<br>

In [22]:
# Create an empty list
cover_list = []

# For subset sizes ranging from 1 to the number of items minus 1
for subset_size in range(1, len(items)):
    
    # Find all subsets of the current size
    subset_object = findsubsets(items,subset_size)
    
    # For each subset
    for current_subset in subset_object:
        
        # Initialize a temporary variable tracking capacity to zero
        capacity_used = 0
        
        # For all indices of the current subset
        for i in range(subset_size):
            
            # Increment the capacity tracking variable by the 
            # weight of the item associated with the current index
            capacity_used += weights[current_subset[i]]
            
            # If the capacity is greater than the available capacity
            if capacity_used > available_capacity:
                
                # Add the subset to the list
                cover_list.append(current_subset)

# Print the first five subsets in the list
cover_list[:5]

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 3)]

The following code block defines the callback function that we will use. **Note that the *where* condition indicates that the solver is processing a new node, not that a new solution has been found**.

[Back to Table of Contents](#Table_of_Contents)<br>

In [23]:
def add_cover_cuts(model, where):
    
    # If a new node is being processed
    if where == GRB.Callback.MIPNODE:    
        
        # Implement the subset construction method previously shown
        cover_list = []
        for subset_size in range(1, len(items)):
            subset_object = findsubsets(items,subset_size)
            for current_subset in subset_object:
                capacity_used = 0
                for i in range(subset_size):
                    capacity_used += weights[current_subset[i]]
                    if capacity_used > available_capacity:
                        cover_list.append(current_subset)
        
        # For each subset in the list
        for current_cover in cover_list:
            
            # Initialize an empty linear expression
            temp_sum = LinExpr()
            
            # Initialize a temporary counter to zero
            count = 0
            
            # For each index in the current subset
            for i in range(len(current_cover)):
                
                # Add the assignment variable for the index to the Linear expression
                temp_sum.add(model._vars[current_cover[i]])
                
                # Increment the count by 1
                count += 1
            
            # Add a cut that requires the sum captured by the linear expression to be less than
            # or equal to the value of the counter minus one.
            model.cbCut(temp_sum <= count-1)

The following code block solves the problem using the cover inequality callback. **Note the number of nodes that were considered.**

[Back to Table of Contents](#Table_of_Contents)<br>

In [24]:
model_build_starttime = datetime.datetime.now()

model = Model('Knapsack')
model.setParam('PreCrush', 1)
model.setParam('Heuristics', 0)
model.Params.Presolve = 0
model.Params.Threads = 1
model.Params.Cuts = 0

assignment = model.addVars(items,
                           vtype=GRB.BINARY,
                           name="Item")

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*values[i])
model.setObjective(temp_sum, GRB.MAXIMIZE)

temp_sum = LinExpr()
for i in items:
    temp_sum.add(assignment[i]*weights[i])
model.addConstr(temp_sum <= available_capacity)

model._vars = assignment
model.optimize(add_cover_cuts)
#model.optimize()

assigned_items = []
for current_item in range(len(assignment)):
    if assignment[current_item].x > 0.5:
        assigned_items.append(assignment[current_item].varname)

model_solve_time = (datetime.datetime.now()-model_build_starttime).total_seconds()

print(f'\nThe model solved in {model_solve_time} seconds with an optimal objective of {model.objVal}.\n')
print(f'The optimal solution chooses:\n {assigned_items}')

Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 1 rows, 7 columns and 7 nonzeros
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Variable types: 0 continuous, 7 integer (7 binary)

Root relaxation: objective 1.727273e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   17.27273    0    1          -

This concludes the demonstration.

[Back to Table of Contents](#Table_of_Contents)<br>